In [2]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

## Image Data Agumentation

In [4]:
#setting parameter for Image Data agumentation to the traing data
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [5]:
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

## Loading our data and performing data agumentation

In [6]:
#performing data agumentation to train data
x_train=train_datagen.flow_from_directory(directory=r'D:\personal\ECG\ECG-main\data\train'
                                          ,target_size=(64,64),batch_size=32,class_mode='categorical')
#performing data agumentation to test data
x_test=test_datagen.flow_from_directory(directory=r'D:\personal\ECG\ECG-main\data\test'
                                        ,target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [7]:
print(x_train.class_indices)#checking the number of classes

{'Left Bundle Branch Block': 0, 'Normal': 1, 'Premature Atrial Contraction': 2, 'Premature Ventricular Contractions': 3, 'Right Bundle Branch Block': 4, 'Ventricular Fibrillation': 5}


In [8]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 504, 1: 7346, 2: 2054, 3: 2759, 4: 2239, 5: 439})

## Creating the model

In [9]:
# create model
model=Sequential()
# adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))#convolutional layer
model.add(MaxPooling2D(pool_size=(2,2))) #MaxPooling2D-for downsampling the input

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(32))#deeply connected neural network layers.
model.add(Dense(6,activation='softmax'))#output layer with 6 neurons


In [10]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                200736    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

In [11]:
# Compile model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),
                    epochs=10, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/10
480/480 [==============================] - 461s 960ms/step - loss: 0.7830 - accuracy: 0.7351 - val_loss: 0.6307 - val_accuracy: 0.8149
Epoch 2/10
480/480 [==============================] - 116s 241ms/step - loss: 0.2845 - accuracy: 0.9167 - val_loss: 0.3774 - val_accuracy: 0.8999
Epoch 3/10
480/480 [==============================] - 116s 243ms/step - loss: 0.2171 - accuracy: 0.9375 - val_loss: 0.2908 - val_accuracy: 0.8949
Epoch 4/10
480/480 [==============================] - 116s 242ms/step - loss: 0.1754 - accuracy: 0.9483 - val_loss: 0.3944 - val_accuracy: 0.8709
Epoch 5/10
480/480 [==============================] - 117s 243ms/step - loss: 0.1498 - accuracy: 0.9549 - val_loss: 0.2788 - val_accuracy: 0.9026
Epoch 6/10
480/480 [==============================] - 117s 243ms/step - loss: 0.1322 - accuracy: 0.9602 - val_loss: 0.2980 - val_accuracy: 0.9068
Epoch 7/10
480/480 [==============================] - 117s 245ms/step - loss: 0.1184 - accuracy: 0.9627 - val_loss: 0.3037 -

In [16]:
# Save the model
model.save('ECG.h5')

In [17]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("ECG.h5") #loading the model for testing

In [21]:
img = image.load_img(r"D:\personal\ECG\ECG-main\Flask\uploads\PAC.png",target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([2], dtype=int64)

In [22]:
index=['Left Bundle Branch Block','Normal','Premature Atrial Contraction',
       'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[pred[0]])
result

'Premature Atrial Contraction'